In [2]:
from websocket import WebSocket
import json
import pytest
import numpy as np
from rasa_core.envconfig import EnvConfig

In [3]:
env_config = EnvConfig("../envconfig.json")

In [4]:
sender_id = str(np.random.randint(123456))+"_"+"john123"

In [5]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
def login(idd):
    return json.dumps({'status':'login','sender':idd})

def send_msg(idd,message):
    return json.dumps({'text':message,'sender':idd})

In [7]:
from websocket import WebSocket
import json
import pytest
from rasa_core.envconfig import EnvConfig
import numpy as np
import requests

env_config = EnvConfig("../envconfig.json")



ws_user = WebSocket()
ws_agent = WebSocket()




def test_connection():
    ws_user.connect('ws://0.0.0.0:'+env_config.userchannelport+'/webhook')
    ws_agent.connect('ws://0.0.0.0:'+env_config.hachannelport+'/webhook')
    ws_user.send(login(sender_id))

    
    ws_user.send(send_msg(sender_id,'Hi'))
    received_data=ws_user.recv()


    ws_user.send(send_msg(sender_id,'Yes'))
    received_data=ws_user.recv()

    
    ws_agent.send(login('HA_agent5+'+sender_id))
    received_data=ws_agent.recv()

    
    ws_user.send(send_msg(sender_id,'Yes'))
    received_data=ws_user.recv()
    
    ws_user.send(send_msg(sender_id,'ok'))
    received_data=ws_user.recv()
    
    received_data = requests.get("http://localhost:6212/?sender_id="+sender_id)
    ws_agent.send(send_msg('HA_agent5+'+sender_id, "Hi john, Your payoff amount is $1461"))

    received_data=ws_user.recv()
    ws_user.send(send_msg(sender_id,"ok"))
             
    


In [8]:
test_connection()

In [ ]:
env_config.hachannelport

In [ ]:
r = requests.get("http://localhost:6212/?sender_id="+sender_id)

In [ ]:
r.json()

In [ ]:
import requests

In [12]:
import redis
import _pickle
a = redis.StrictRedis(host = "idaredb.eastus.cloudapp.azure.com", port =6379)
tracker = a.get("e5dz24zs714b13guf2eyu1bj28z2g3pq_john1234")
tracker = _pickle.loads(tracker)

In [13]:
tracker.events[-1].message_data

{'text': 'okay',
 'intent': 'affirm  ',
 'confidence': '0.93',
 'recom': ['Hi. Your Payoff amount is $6973',
  'Hey there. Your Payoff amount is $6973',
  'Hi. Sorry for the inconvenience caused. Your Payoff amount is $6973'],
 'links': ['http://idare1.eastus.cloudapp.azure.com:8081/static/pdf/balance_calc.pdf',
  'http://idare1.eastus.cloudapp.azure.com:8081/static/images/payoff_calc.png',
  'http://idare1.eastus.cloudapp.azure.com:8081/static/pdf/Mortgage_servicing.pdf'],
 'context_field': ['customerid',
  'loanamount',
  'loanterm',
  'rate',
  'paymentsmade'],
 'context_info': ['john1234', '100000.00', '30', '4.00', '345'],
 'insights': ['The payoff amount is calculated using the formulae',
  'The payoff amount is calculated using the formulae',
  'The payoff amount is calculated using the formulae'],
 'sender_id': 'e5dz24zs714b13guf2eyu1bj28z2g3pq_john1234',
 'chat_open_time': 1537528294.4074605,
 'topic': 'payoff',
 'topic_confidence': '1.00',
 'event_id': 'assist_1537528301.2379

In [ ]:
 received_data = requests.get("http://localhost:6212/?sender_id="+sender_id)
    print("recommended answer : ")
    print(received_data.json()["requests"]["recom"][1])
    ws_agent.send(send_msg('HA_agent5+'+sender_id,"Your payoff fee has also been waived off"))
    assert(received_data.json()["requests"]["intent"]=="affirm  ")
    
    

    
    ws_user.send(send_msg(sender_id,"ok"))
    received_data=ws_user.recv()
    print(received_data)
    print("-"*20)

    '''
    
    received_data = requests.get("http://localhost:6212/?sender_id="+sender_id)
    ws_agent.send(send_msg('HA_agent5+'+sender_id,received_data.json()["requests"]["recom"][1]))
    assert(received_data.json()["requests"]["intent"]=="affirm  ")
    
    

    ws_agent.send(send_msg('HA_agent5+'+sender_id,'How would you like your quote to be delivered to you?'))
    received_data=ws_agent.recv()
    assert(received_data.json()["requests"]["intent"]=="affirm  ")
    print("-"*20)
    
    
    ws_user.send(send_msg(sender_id,"I would like it mailed to me"))
    received_data=ws_user.recv()
    print(received_data)
    print("-"*20)
    
    ws_agent.send(send_msg('HA_agent5+'+sender_id,'How would you like your quote to be delivered to you?'))
    received_data=ws_agent.recv()
    assert(received_data.json()["requests"]["intent"]=="affirm  ")
    print("-"*20)
    
    ws_user.send(send_msg(sender_id,"Okay thanks"))
    received_data=ws_user.recv()
    print(received_data)
    
    ws_agent.send(send_msg('HA_agent5+'+sender_id,'Is there anything else ?'))
    received_data=ws_agent.recv()
    print(received_data)
    '''

In [ ]:
from rasa_nlu.model import Metadata, Interpreter
interpreter = Interpreter.load('../nlu_model/default/current')

In [ ]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [ ]:
interp_data = interpreter.parse('Hi')
assert(interp_data['act']['name'] == 'greet')

interp_data = interpreter.parse('Hi, are you here for a Payoff quote?')
assert(interp_data['act']['name'] == 'confirm' and interp_data['slots'][0]['entity'] == 'payoff')

interp_data = interpreter.parse('yes')
assert(interp_data['act']['name'] == 'affirm')

interp_data = interpreter.parse('Sorry we could not resolve your Payoff request earlier. I will transfer you to a Payoff specialist who can assist you with this. Is that OK?')
assert(interp_data['act']['name'] == 'request' and interp_data['slots'][0]['entity'] == 'request_route')

interp_data = interpreter.parse('ok')
assert(interp_data['act']['name'] == 'affirm')

interp_data = interpreter.parse('Hi there. Your payoff amount is $1351.')
assert(interp_data['act']['name'] == 'inform' and interp_data['slots'][0]['entity'] == 'payoff_amount')

interp_data = interpreter.parse('Is there a payoff fee')
assert(interp_data['act']['name'] == 'inform' and interp_data['slots'][0]['entity'] == 'penalty')

interp_data = interpreter.parse('The Payoff fee has also been waived off . Do you wish to go ahead with the Payoff ?')
assert(interp_data['act']['name'] == 'inform' and interp_data['slots'][0]['entity'] == 'waive_off')

interp_data = interpreter.parse('Ok thats great !')
assert(interp_data['act']['name'] == 'affirm')

interp_data = interpreter.parse('How would you like your quote to be delivered to you?')
assert(interp_data['act']['name'] == 'request' and interp_data['slots'][0]['entity'] == 'deliver')

interp_data = interpreter.parse('I want it mailed to me')
assert(interp_data['act']['name'] == 'inform' and interp_data['slots'][0]['entity'] == 'delivery_mode')

interp_data = interpreter.parse('Alright. We will have it delivered in 6-7 days')
assert interp_data['act']['name'] == 'affirm' and interp_data['slots'][0]['entity'] == 'deliver_days'



In [ ]:
interp_data['text']

In [ ]:
f'Text was { interp_data }'